# Versatility of GPT-2 for Tasks in other Domains:

tasks: Bit XOR, Bit Memory

In [ ]:
#@title Install Package
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.7 MB/s eta 0:00:00


In [ ]:
#@title Import Package
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import numpy as np
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
from typing import List, Dict
from tqdm import tqdm
import matplotlib.pyplot as plt
import requests
from PIL import Image
import cv2

%matplotlib inline 

## Task 1: Bit-wise operation

### part A: Creating the dataset

Simple logic of the generate the data

In [ ]:
if torch.cuda.is_available():
   device = 'cuda'
else:
  device = 'cpu'

In [ ]:
# randomly generate two n-bits strings and its ground-truth and result
def generate_example(n):
  bits = np.random.randint(low=0, high=2, size=(2, n)) 
  XOR = np.logical_xor(bits[0], bits[1]).astype(np.long) 
  return bits.reshape((2*n)), XOR

In [ ]:
n=5
bits, XOR = generate_example(n)
print('  String 1:', bits[:n])
print('  String 2:', bits[n:])
print('Output XOR:', XOR)

  String 1: [0 1 1 0 1]
  String 2: [1 0 1 0 0]
Output AND: [1 1 0 0 1]


<ipython-input-6-92986c992621>:4: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  And = np.logical_xor(bits[0], bits[1]).astype(np.long)


Create the dataset

In [ ]:
class BitWiseDataset(torch.utils.data.Dataset):
  def __init__(self, n, size):
    self.n = n
    self.size = size

  def __len__(self):
    return self.size

  def __getitem__(self, idx):
    bits = np.random.randint(low=0, high=2, size=(2, self.n))
    And = np.logical_xor(bits[0], bits[1]).astype(np.int64)
    return torch.tensor(bits.reshape((2*self.n)), dtype=torch.long).to(device), torch.tensor(And, dtype=torch.long).to(device)

def generate_data_loaders(n, batch_size, data_size = 1000, train_size=0.8):
  dataset = BitWiseDataset(n, size=data_size)
  train_size = int(train_size * len(dataset))
  test_size = len(dataset) - train_size
  train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, test_loader


### part B: Modify and train the pretained transformer to our specific tasks
Load the pretain transformer GPT2-model and freeze the weights of the self-attention and feedforward layers.



In [ ]:
# load the GP2 model
gpt2 = GPT2Model.from_pretrained('gpt2')

In [ ]:
# show the name of all the para your are able to modified in this model
for name, param in gpt2.named_parameters():
  print(name)

wte.weight
wpe.weight
h.0.ln_1.weight
h.0.ln_1.bias
h.0.attn.c_attn.weight
h.0.attn.c_attn.bias
h.0.attn.c_proj.weight
h.0.attn.c_proj.bias
h.0.ln_2.weight
h.0.ln_2.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_proj.weight
h.0.mlp.c_proj.bias
h.1.ln_1.weight
h.1.ln_1.bias
h.1.attn.c_attn.weight
h.1.attn.c_attn.bias
h.1.attn.c_proj.weight
h.1.attn.c_proj.bias
h.1.ln_2.weight
h.1.ln_2.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_proj.weight
h.1.mlp.c_proj.bias
h.2.ln_1.weight
h.2.ln_1.bias
h.2.attn.c_attn.weight
h.2.attn.c_attn.bias
h.2.attn.c_proj.weight
h.2.attn.c_proj.bias
h.2.ln_2.weight
h.2.ln_2.bias
h.2.mlp.c_fc.weight
h.2.mlp.c_fc.bias
h.2.mlp.c_proj.weight
h.2.mlp.c_proj.bias
h.3.ln_1.weight
h.3.ln_1.bias
h.3.attn.c_attn.weight
h.3.attn.c_attn.bias
h.3.attn.c_proj.weight
h.3.attn.c_proj.bias
h.3.ln_2.weight
h.3.ln_2.bias
h.3.mlp.c_fc.weight
h.3.mlp.c_fc.bias
h.3.mlp.c_proj.weight
h.3.mlp.c_proj.bias
h.4.ln_1.weight
h.4.ln_1.bias
h.4.attn.c_attn.weight
h.4.attn.c_at

In [ ]:
for name, param in gpt2.named_parameters():
# freeze all parameters except the layernorm and positional embeddings 
  if 'ln' in name or 'wpe' in name:
    param.requires_grad = True 
  else:
    param.requires_grad = False

Design and train your own network using pretrained GPT2 model as universal computational engine. You need to design

*   An input embedding layer, which projects your input to accommodate GPT2 input demension

*   An output Linear layer and loss function

In [ ]:
class Bit_wise_transformer(nn.Module):
  def __init__(self, engine, bitLength, input_dim, engine_embed_dim, n_class = 2):
    super().__init__()
    self.n = bitLength
    self.input_embed = nn.Embedding(input_dim, engine_embed_dim)
    self.engine = engine
    self.output_layer = nn.Linear(engine_embed_dim, n_class)
  def forward(self, x):
    embeddings = self.input_embed(x)
    hidden_state = self.engine(inputs_embeds=embeddings).last_hidden_state[:,self.n:]
    logits = self.output_layer(hidden_state)[0]
    return logits

In [ ]:
# generate the training and testing data
train_loader, test_loader = generate_data_loaders(n=5, batch_size=1,data_size = 1000, train_size=0.8)

In [ ]:
#@title train the modified model
# create an instance of the model
model = Bit_wise_transformer(
      gpt2, 
      bitLength = Bit_length,
      input_dim = 2, 
      engine_embed_dim = 768
).to(device)


# define the optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


losses = []
train_acc = []
all_val_acc = []
best_val_acc = 0
num_epochs = 5

epoch_iterator = trange(num_epochs)
for epoch in epoch_iterator:
    # Training loop
    running_loss = 0.0
    data_iterator = tqdm(train_loader)
    for i, (inputs, labels) in enumerate(data_iterator):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.squeeze())
        accuracy = torch.mean((torch.argmax(outputs, dim=-1) == labels.flatten()).float())
        loss.backward()
        optimizer.step()

        data_iterator.set_postfix(loss=loss.item())

        running_loss += loss.item()
        if (i + 1) % 10 == 0:
              #print(f'Epoch [{epoch + 1}/{num_epochs}], Data [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0
            losses.append(loss.item())
            train_acc.append(accuracy.item())
        
    # Validation
    val_acc = []
    model.eval()
    with torch.no_grad():
      for inputs, labels in test_loader:
          inputs = inputs.to(device=device, dtype=torch.long)
          labels = labels.to(device=device, dtype=torch.long)
          outputs = model(inputs)
          accuracy = torch.mean((torch.argmax(outputs, dim=-1) == labels.flatten()).float())
          val_acc.append(accuracy.item())
    model.train()

    all_val_acc.append(np.mean(val_acc))
    # Save best model
    if np.mean(val_acc) > best_val_acc:
        best_val_acc = np.mean(val_acc)

    epoch_iterator.set_postfix(val_acc=np.mean(val_acc), best_val_acc=best_val_acc)


plt.plot(losses)
plt.title('Train Loss')
plt.figure()
plt.plot(train_acc)
plt.title('Train Accuracy')
plt.figure()
plt.plot(all_val_acc)
plt.title('Val Accuracy')
del model
torch.cuda.empty_cache()

<ipython-input-8-e37a48aed99d>:11: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  And = np.logical_and(bits[0], bits[1]).astype(np.long)


Epoch [1/5], Batch [100/800], Loss: 0.3933
Epoch [1/5], Batch [200/800], Loss: 0.3741
Epoch [1/5], Batch [300/800], Loss: 0.3219
Epoch [1/5], Batch [400/800], Loss: 0.3138
Epoch [1/5], Batch [500/800], Loss: 0.2900
Epoch [1/5], Batch [600/800], Loss: 0.2794
Epoch [1/5], Batch [700/800], Loss: 0.2708
Epoch [1/5], Batch [800/800], Loss: 0.2949
Epoch [2/5], Batch [100/800], Loss: 0.2990
Epoch [2/5], Batch [200/800], Loss: 0.2838
Epoch [2/5], Batch [300/800], Loss: 0.2754
Epoch [2/5], Batch [400/800], Loss: 0.2564
Epoch [2/5], Batch [500/800], Loss: 0.2650
Epoch [2/5], Batch [600/800], Loss: 0.2348
Epoch [2/5], Batch [700/800], Loss: 0.2304
Epoch [2/5], Batch [800/800], Loss: 0.2433
Epoch [3/5], Batch [100/800], Loss: 0.2076
Epoch [3/5], Batch [200/800], Loss: 0.2196
Epoch [3/5], Batch [300/800], Loss: 0.2113
Epoch [3/5], Batch [400/800], Loss: 0.2883
Epoch [3/5], Batch [500/800], Loss: 0.2538
Epoch [3/5], Batch [600/800], Loss: 0.2374
Epoch [3/5], Batch [700/800], Loss: 0.2331
Epoch [3/5]

## Task 2: Bit Memory.

### part A: Loading the dataset

Simple logic of the generate the data

In [ ]:
if torch.cuda.is_available():
   device = 'cuda'
else:
  device = 'cpu'

Create the dataset

In [ ]:
class Dataset:

    def __init__(self, device='cpu'):
        self.device = device
        self._ind = 0

    def get_batch(self, batch_size, train=True):
        x, y = self.get_batch_np(batch_size, train=train)
        x = torch.from_numpy(x).to(device=self.device, dtype=torch.float32)
        y = torch.from_numpy(y).to(device=self.device, dtype=torch.long)
        self._ind += 1
        return x, y

    def get_batch_np(self, batch_size, train):
        raise NotImplementedError

    def start_epoch(self):
        self._ind = 0

In [ ]:
class BitMemoryDataset(Dataset):

    def __init__(self, n=1000, num_patterns=5, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n = n
        self.num_patterns = num_patterns

    def get_batch_np(self, batch_size, train):
        bits = np.random.randint(low=0, high=2, size=(batch_size, self.num_patterns, self.n))
        bits = 2 * bits - 1
        query_inds = np.random.randint(low=0, high=self.num_patterns, size=batch_size)
        query_bits = bits[range(batch_size), query_inds]
        mask = np.random.randint(low=0, high=2, size=query_bits.shape)
        masked_query_bits = mask * query_bits
        masked_query_bits = masked_query_bits.reshape(batch_size, 1, self.n)
        x = np.concatenate([bits, masked_query_bits], axis=1)
        y = query_bits
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

In [ ]:
class BitMemoryDataset(torch.utils.data.Dataset):
  def __init__(self,bit_length =100, num_patterns=5, size = 100):
    self.bit_length = bit_length
    self.num_patterns = num_patterns
    self.size = size

  def __len__(self):
    return self.size

  def __getitem__(self, idx):
    bits = np.random.randint(low=0, high=2, size=(1, self.num_patterns, self.bit_length))
    bits = 2 * bits - 1
    query_inds = np.random.randint(low=0, high=self.num_patterns, size=1)
    query_bits = bits[range(1), query_inds]
    mask = np.random.randint(low=1, high=2, size=query_bits.shape)
    masked_query_bits = mask * query_bits
    masked_query_bits = masked_query_bits.reshape(1, 1, self.bit_length)
    x = np.concatenate([bits, masked_query_bits], axis=1)
    y = query_bits

    return torch.tensor(x, dtype=torch.float32).squeeze(), torch.tensor(y, dtype=torch.long).squeeze()



def generate_data_loaders(size = 100, batch_size = 1, train_size=0.8):
  dataset = BitMemoryDataset(size=size)
  train_size = int(train_size * len(dataset))
  test_size = len(dataset) - train_size
  train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, test_loader

### part B: Modified the pretained transformer to our specific tasks
Load the pretain transformer GPT2-model and freeze the weights of the self-attention and feedforward layers.



In [ ]:
# load the GP2 model
gpt2 = GPT2Model.from_pretrained('gpt2')

In [ ]:
# show the name of all the para your are able to modified in this model
for name, param in gpt2.named_parameters():
  print(name)

wte.weight
wpe.weight
h.0.ln_1.weight
h.0.ln_1.bias
h.0.attn.c_attn.weight
h.0.attn.c_attn.bias
h.0.attn.c_proj.weight
h.0.attn.c_proj.bias
h.0.ln_2.weight
h.0.ln_2.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_proj.weight
h.0.mlp.c_proj.bias
h.1.ln_1.weight
h.1.ln_1.bias
h.1.attn.c_attn.weight
h.1.attn.c_attn.bias
h.1.attn.c_proj.weight
h.1.attn.c_proj.bias
h.1.ln_2.weight
h.1.ln_2.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_proj.weight
h.1.mlp.c_proj.bias
h.2.ln_1.weight
h.2.ln_1.bias
h.2.attn.c_attn.weight
h.2.attn.c_attn.bias
h.2.attn.c_proj.weight
h.2.attn.c_proj.bias
h.2.ln_2.weight
h.2.ln_2.bias
h.2.mlp.c_fc.weight
h.2.mlp.c_fc.bias
h.2.mlp.c_proj.weight
h.2.mlp.c_proj.bias
h.3.ln_1.weight
h.3.ln_1.bias
h.3.attn.c_attn.weight
h.3.attn.c_attn.bias
h.3.attn.c_proj.weight
h.3.attn.c_proj.bias
h.3.ln_2.weight
h.3.ln_2.bias
h.3.mlp.c_fc.weight
h.3.mlp.c_fc.bias
h.3.mlp.c_proj.weight
h.3.mlp.c_proj.bias
h.4.ln_1.weight
h.4.ln_1.bias
h.4.attn.c_attn.weight
h.4.attn.c_at

In [ ]:
for name, param in gpt2.named_parameters():
# freeze all parameters except the layernorm and positional embeddings 
  if 'ln' in name or 'wpe' in name:
    param.requires_grad = True 
  else:
    param.requires_grad = False

Design and train your own network using pretrained GPT2 model as universal computational engine. You need to design

*   An input embedding layer, which projects your input to accommodate GPT2 input demension

*   An output Linear layer and loss function

In [ ]:
class Bit_Memory_transformer(nn.Module):
  def __init__(self, engine,input_dim, engine_embed_dim, n_class, patch_size):
    super().__init__()
    self.input_dim = input_dim
    self.output_dim = n_class
    self.input_embed = nn.Linear(input_dim, engine_embed_dim)
    self.engine = engine
    self.output_layer = nn.Linear(engine_embed_dim, n_class)
  def forward(self, x):
        orig_dim = x.shape[-1]
        ratio = orig_dim // self.input_dim
        x = x.reshape(x.shape[0], x.shape[1] * ratio, self.input_dim)
        flattened_input = x.view(-1, 50)

        x = self.input_embed(flattened_input)
        
        x = self.engine(inputs_embeds = x.unsqueeze(0)).last_hidden_state
        x = x[:,-ratio:]
        x = self.output_layer(x)

        x = x.reshape(x.shape[0], x.shape[1] // ratio, ratio * self.output_dim)
        return x
        

    

In [ ]:
ce_loss = torch.nn.CrossEntropyLoss()
task = 'bit-memory'
n = 100
def loss_fn(out, y, x=None):
  out = torch.reshape(out, (-1, n, 2))
  ids = torch.zeros(y.shape).to(device=y.device).long()
  if task == 'bit-memory':
      ids[y < 0], ids[y > 0] = 0, 1
  else:
      ids[y < 0.5], ids[y > 0.5] = 0, 1
  out, ids = torch.reshape(out, (-1, 2)), torch.reshape(ids, (-1,))
  return ce_loss(out, ids)

def accuracy_fn(preds, true, x=None):
    if task == 'bit-memory':
        preds = preds.reshape(-1, n, 2).argmax(-1) * 2 - 1
    else:
        preds = preds.reshape(-1, n, 2).argmax(-1)
    if task == 'bit-memory':
        return (np.sign(preds) == np.sign(true)).mean()
    else:
        return ((preds > 0.5) == (true > 0.5)).mean()

In [ ]:
model = Bit_Memory_transformer(
    gpt2, 
    input_dim = 50, 
    engine_embed_dim = 768,
    n_class= 2*50,
    patch_size = 50
)

def get_loss(x,y,return_acc = True):
  out = model(x)
  loss = loss_fn(out, y, x=x)
  accs = accuracy_fn(
                out.detach().cpu().numpy(),
                y.detach().cpu().numpy(),
                x=x.detach().cpu().numpy(),
            )
  return loss, accs

train_loader, test_loader = generate_data_loaders(size = 1000, batch_size=1, train_size=0.8)

num_epochs = 3
for epoch in range(num_epochs):
    # Training loop
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
      loss, acc = get_loss(inputs, labels, return_acc=True)
      loss.backward()
      running_loss += loss.detach().cpu().item()
      optimizer.step()
      optimizer.zero_grad()
      if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0


Epoch [1/3], Batch [100/800], Loss: 1.7978
Epoch [1/3], Batch [200/800], Loss: 1.8176
Epoch [1/3], Batch [300/800], Loss: 1.8573
Epoch [1/3], Batch [400/800], Loss: 1.7865
Epoch [1/3], Batch [500/800], Loss: 1.8767
Epoch [1/3], Batch [600/800], Loss: 1.9043
Epoch [1/3], Batch [700/800], Loss: 1.8139
Epoch [1/3], Batch [800/800], Loss: 1.8507
Epoch [2/3], Batch [100/800], Loss: 1.8376
Epoch [2/3], Batch [200/800], Loss: 1.8109
Epoch [2/3], Batch [300/800], Loss: 1.8670
Epoch [2/3], Batch [400/800], Loss: 1.8446
Epoch [2/3], Batch [500/800], Loss: 1.8444
Epoch [2/3], Batch [600/800], Loss: 1.8390
Epoch [2/3], Batch [700/800], Loss: 1.8304
Epoch [2/3], Batch [800/800], Loss: 1.8627
Epoch [3/3], Batch [100/800], Loss: 1.8082
Epoch [3/3], Batch [200/800], Loss: 1.8063
Epoch [3/3], Batch [300/800], Loss: 1.8468
Epoch [3/3], Batch [400/800], Loss: 1.8781
Epoch [3/3], Batch [500/800], Loss: 1.8946
Epoch [3/3], Batch [600/800], Loss: 1.8022
Epoch [3/3], Batch [700/800], Loss: 1.8479
Epoch [3/3]

In [ ]:
   # Testing loop
model.eval()
test_loss = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        loss, acc = get_loss(inputs, labels, return_acc=True)
        test_loss += loss.detach().cpu().item() / len(test_loader)
        accuracy += acc / len(test_loader)

print(len(test_loader))
print(f'Testing accuracy: {100 * accuracy} %')